In [1]:
!pip install pandas openpyxl

In [462]:
import pandas as pd

In [463]:
file_path = "TJA_MyChartMessages.xlsx"
xls = pd.read_excel(file_path, sheet_name=None, engine='openpyxl')

In [464]:
sheet1 = xls['Sheet1']
sheet2 = xls['3WeeksPrior']
sheet3 = xls['90DayPost']
sheet4 = xls['PRO']

sheet1.head()
sheet1.dtypes

In [466]:
column_names = {
    'PrimaryMrn': 'mrn',
    'MyChartStatus': 'mychart_status',
    'Age': 'age',
    'BirthDate': 'date_birth',
    'Sex': 'sex',
    'Race': 'race',
    'Ethnicity': 'ethnicity',
    'PreferredLanguage': 'preferred_language',
    'PostalCode': 'postal_code',
    'MaritalStatus': 'marital_status',
    'BenefitPlanName': 'payer_name',
    'PayorFinancialClass': 'payer_type',
    'Asa': 'asa',
    'SurgeonName': 'surgeon_name',
    'ProcedureName': 'procedure',
    'AdmissionPatientClass': 'admission_class',
    'Admitdate': 'date_admit',
    'SurgeryDate': 'date_surgery',
    'ProcStartTime': 'time_proc_start',
    'ProcEndTime': 'time_proc_end',
    'DischargeDate': 'date_discharge',
    'LengthOfStayHours': 'los_hours',
    'PrimaryProcedureLaterality': 'laterality',
    'PrimaryAnesthesiaType': 'anesthesia_type',
    'DischargeDisposition': 'discharge_disposition',
}

sheet1.rename(columns=column_names, inplace=True)

In [467]:
sheet1['postal_code'] = sheet1['postal_code'].astype(str)
sheet1['five_digit_postal_code'] = sheet1['postal_code'].str.slice(0, 5)
sheet1 = sheet1.drop('postal_code', axis=1)
sheet1['english_preferred_language'] = sheet1['preferred_language'].apply(lambda x: 1 if x == 'English' else 0)
def simplify_race(race):
    white = ['White']
    black_african_american = ['Black, African American']
    asian = [
        'Chinese', 'Asian Indian', 'Asian - unspecified',
        'Filipino', 'Asian', 'Pakistani', 'Korean', 'Japanese',
        'Bangladeshi', 'Laotian', 'Thai', 'Indonesian', 'Sri Lankan',
        'Cambodian', 'Vietnamese'
    ]
    american_indian = ['Native American (American Indian/Eskimo/Aleutian)']
    pacific_islander = ['Other Pacific Islander', 'Guamanian or Chamorro']
    other = ['Other Race']
    unknown = ['Unknown']
    patient_refused = ['Patient Refused']

    if race in white:
        return 'white'
    elif race in black_african_american:
        return 'black or african american'
    elif race in asian:
        return 'asian'
    elif race in american_indian:
        return 'american indian'
    elif race in pacific_islander:
        return 'native hawaiian or other pacific islander'
    elif race in other:
        return 'other'
    elif race in unknown:
        return 'unknown'
    elif race in patient_refused:
        return 'patient refused'
    else:
        return 'unknown'
sheet1['race_simplified'] = sheet1['race'].apply(simplify_race)






In [468]:
mrn_counts = sheet1['mrn'].value_counts()
frequency_counts = mrn_counts.value_counts().sort_index()
print(frequency_counts)
# Filter the MRNs that appear 4 times
mrns_with_4_occurrences = mrn_counts[mrn_counts == 4].index

# Get all rows in the DataFrame associated with the MRNs that show up 4 times
rows_with_4_occurrences = sheet1[sheet1['mrn'].isin(mrns_with_4_occurrences)]

# Select the desired columns
desired_columns = ['mrn', 'procedure', 'date_admit', 'date_surgery', 'date_discharge', 'laterality']
filtered_rows_with_4_occurrences = rows_with_4_occurrences[desired_columns]

# Display the rows with the desired columns








1    15416
2     2533
3       62
4        7
Name: mrn, dtype: int64


In [469]:
# Convert the 'mrn' and 'date_surgery' columns to strings
sheet1['mrn'] = sheet1['mrn'].astype(str)
sheet1['date_surgery'] = sheet1['date_surgery'].astype(str)

# Create a new column by concatenating the 'mrn' and 'date_surgery' columns
sheet1['mrn_date_surgery'] = sheet1['mrn'] + '_' + sheet1['date_surgery']

# Display the DataFrame with the new 'mrn_date_surgery' column
print(sheet1.head())


     mrn mychart_status  age date_birth     sex   race  \
0  10847      Activated   79 1941-09-15    Male  White   
1  14975      Activated   81 1938-12-01    Male  White   
2  14975      Activated   82 1938-12-01    Male  White   
3  21400      Activated   80 1939-01-03  Female  White   
4  23732      Activated   85 1933-10-28    Male  White   

                        ethnicity preferred_language marital_status  \
0                    *Unspecified            English        Married   
1  Not of Spanish/Hispanic Origin            English        Married   
2  Not of Spanish/Hispanic Origin            English        Married   
3  Not of Spanish/Hispanic Origin            English        Widowed   
4                    *Unspecified            English         Single   

              payer_name  ... time_proc_end date_discharge los_hours  \
0  MEDICARE PART A AND B  ...      15:40:00     2021-02-11        53   
1  MEDICARE PART A AND B  ...      12:06:00     2020-01-17        29   
2  MEDIC

In [470]:
import pandas as pd

variables = [
    'mychart_status', 'sex', 'race', 'ethnicity', 'preferred_language',
    'marital_status', 'payer_name', 'payer_type', 'asa', 'surgeon_name',
    'procedure', 'admission_class', 'laterality', 'anesthesia_type', 'discharge_disposition'
]

unique_values_data = []

for variable in variables:
    value_counts = sheet1[variable].value_counts().reset_index()
    value_counts.columns = ['Value', 'Count']
    unique_values_data.append(value_counts)

unique_values_df = pd.concat(unique_values_data, keys=variables, axis=1)
unique_values_df = unique_values_df.fillna(value=pd.NA)


In [471]:
# Reset the MultiIndex columns to a single level
unique_values_df.columns = ['_'.join(col) for col in unique_values_df.columns]

# Export the DataFrame to an Excel file
unique_values_df.to_excel('unique_values_report.xlsx', index=False)


In [472]:
categorical_variables = [
    'mychart_status', 'sex', 'race_simplified', 'ethnicity', 'marital_status', 'payer_type',
    'procedure', 'admission_class', 'laterality', 'anesthesia_type', 'discharge_disposition'
]
encoded_data = pd.get_dummies(sheet1, columns=categorical_variables)
sheet1_encoded = pd.get_dummies(sheet1, columns=categorical_variables, prefix=categorical_variables)
sheet1_encoded.to_excel('encoded.xlsx', index=False)

In [473]:
sheet2.head()
sheet2.dtypes

PrimaryMrn             int64
Type                  object
Date          datetime64[ns]
DayOfWeek             object
dtype: object

In [474]:
column_names = {
    'PrimaryMrn': 'mrn',
    'Type': 'type_communication_prior',
    'Date': 'date_communication_prior',
    'DayOfWeek': 'day_communication_prior',
}

sheet2.rename(columns=column_names, inplace=True)

In [475]:
import pandas as pd

# Convert the 'mrn' columns to strings
sheet1['mrn'] = sheet1['mrn'].astype(str)
sheet2['mrn'] = sheet2['mrn'].astype(str)

# Convert date columns to datetime format
sheet1['date_surgery'] = pd.to_datetime(sheet1['date_surgery'])
sheet2['date_communication_prior'] = pd.to_datetime(sheet2['date_communication_prior'])

# Initialize an empty list to store the 'mrn_date_surgery' values for sheet2
sheet2_mrn_date_surgery = []

# Loop through the rows in sheet2
for index, row in sheet2.iterrows():
    mrn2 = row['mrn']
    date_communication_prior = row['date_communication_prior']
    
    # Find the corresponding row in sheet1
    matched_row = sheet1[(sheet1['mrn'] == mrn2) & 
                         (sheet1['date_surgery'] - pd.DateOffset(weeks=3) <= date_communication_prior) &
                         (date_communication_prior <= sheet1['date_surgery'])]
    
    if not matched_row.empty:
        mrn_date_surgery_value = matched_row.iloc[0]['mrn'] + '_' + matched_row.iloc[0]['date_surgery'].strftime('%Y-%m-%d')
    else:
        mrn_date_surgery_value = None
        
    # Append the 'mrn_date_surgery' value to the list
    sheet2_mrn_date_surgery.append(mrn_date_surgery_value)

# Assign the 'mrn_date_surgery' values to the new column in sheet2
sheet2['mrn_date_surgery'] = sheet2_mrn_date_surgery

# Display the updated sheet2 DataFrame
print(sheet2.head())


     mrn type_communication_prior date_communication_prior  \
0  10847          Patient Message               2021-02-05   
1  10847          Patient Message               2021-02-04   
2  10847          Patient Message               2021-02-03   
3  10847          Patient Message               2021-02-02   
4  10847          Patient Message               2021-02-01   

  day_communication_prior  mrn_date_surgery  
0                  Friday  10847_2021-02-09  
1                Thursday  10847_2021-02-09  
2               Wednesday  10847_2021-02-09  
3                 Tuesday  10847_2021-02-09  
4                  Monday  10847_2021-02-09  


In [476]:
variables_sheet2 = [
    'type_communication_prior', 'day_communication_prior'
]

unique_values_data_sheet2 = []

for variable in variables_sheet2:
    value_counts = sheet2[variable].value_counts().reset_index()
    value_counts.columns = ['Value', 'Count']
    unique_values_data_sheet2.append(value_counts)

unique_values_df_sheet2 = pd.concat(unique_values_data_sheet2, keys=variables_sheet2, axis=1)
unique_values_df_sheet2 = unique_values_df_sheet2.fillna(value=pd.NA)

In [477]:
sheet3.head()
sheet3.dtypes

PrimaryMrn             int64
Type                  object
Date          datetime64[ns]
DayOfWeek             object
dtype: object

In [478]:
column_names = {
    'PrimaryMrn': 'mrn',
    'Type': 'type_communication_post',
    'Date': 'date_communication_post',
    'DayOfWeek': 'day_communication_post',
}

sheet3.rename(columns=column_names, inplace=True)

In [479]:
# Convert the 'mrn' column in sheet3 to string
sheet3['mrn'] = sheet3['mrn'].astype(str)

# Convert the 'date_communication_post' column in sheet3 to datetime format
sheet3['date_communication_post'] = pd.to_datetime(sheet3['date_communication_post'])

# Initialize an empty list to store the 'mrn_date_surgery' values for sheet3
sheet3_mrn_date_surgery = []

# Loop through the rows in sheet3
for index, row in sheet3.iterrows():
    mrn3 = row['mrn']
    date_communication_post = row['date_communication_post']
    
    # Find the corresponding row in sheet1
    matched_row = sheet1[(sheet1['mrn'] == mrn3) & 
                         (sheet1['date_surgery'] <= date_communication_post) &
                         (date_communication_post <= sheet1['date_surgery'] + pd.DateOffset(days=90))]
    
    if not matched_row.empty:
        mrn_date_surgery_value = matched_row.iloc[0]['mrn'] + '_' + matched_row.iloc[0]['date_surgery'].strftime('%Y-%m-%d')
    else:
        mrn_date_surgery_value = None
        
    # Append the 'mrn_date_surgery' value to the list
    sheet3_mrn_date_surgery.append(mrn_date_surgery_value)

# Assign the 'mrn_date_surgery' values to the new column in sheet3
sheet3['mrn_date_surgery'] = sheet3_mrn_date_surgery

# Display the updated sheet3 DataFrame
print(sheet3.head())


     mrn type_communication_post date_communication_post  \
0  10847         Patient Message              2021-02-13   
1  10847         Patient Message              2021-02-18   
2  10847         Patient Message              2021-02-19   
3  10847         Patient Message              2021-02-22   
4  10847         Patient Message              2021-03-02   

  day_communication_post  mrn_date_surgery  
0               Saturday  10847_2021-02-09  
1               Thursday  10847_2021-02-09  
2                 Friday  10847_2021-02-09  
3                 Monday  10847_2021-02-09  
4                Tuesday  10847_2021-02-09  


In [480]:
variables_sheet3 = [
    'type_communication_post', 'day_communication_post'
]

unique_values_data_sheet3 = []

for variable in variables_sheet3:
    value_counts = sheet3[variable].value_counts().reset_index()
    value_counts.columns = ['Value', 'Count']
    unique_values_data_sheet3.append(value_counts)

unique_values_df_sheet3 = pd.concat(unique_values_data_sheet3, keys=variables_sheet3, axis=1)
unique_values_df_sheet3 = unique_values_df_sheet3.fillna(value=pd.NA)

In [481]:
# Combine the DataFrames
combined_df = pd.concat([unique_values_df, unique_values_df_sheet2, unique_values_df_sheet3], axis=1)

# Reset the MultiIndex columns to a single level
combined_df.columns = ['_'.join(col) for col in combined_df.columns]

# Export the DataFrame to an Excel file
combined_df.to_excel('combined_unique_values_report.xlsx', index=False)


In [482]:
# Combine the DataFrames
combined_data = pd.concat([sheet1, sheet2[['day_communication_prior']], sheet3[['day_communication_post']]], axis=1)

categorical_variables = [
    'mychart_status', 'sex', 'race_simplified', 'ethnicity', 'marital_status', 'payer_type',
    'procedure', 'admission_class', 'laterality', 'anesthesia_type', 'discharge_disposition',
    'day_communication_prior', 'day_communication_post'
]

# Get dummies for the combined_data
combined_data_encoded = pd.get_dummies(combined_data, columns=categorical_variables, prefix=categorical_variables)

# Export the DataFrame to an Excel file
combined_data_encoded.to_excel('combined_encoded.xlsx', index=False)


In [483]:
sheet4.head()
sheet4.dtypes

MRN                                  int64
SurgeryDate                 datetime64[ns]
VISIT_DATE                  datetime64[ns]
KOOS_JR_L                           object
KOOS_JR_R                           object
HOOS_JR_L                           object
HOOS_JR_R                           object
PROMIS_INTENSITY                    object
PROMIS_INTERFERENCE                 object
PROMIS_PHYSICAL_FUNCTION            object
PROMIS_MOBILITY                    float64
PROMIS_MENTAL_HEALTH                object
PROMIS_PHYSICAL_HEALTH              object
dtype: object

In [484]:
column_names = {
    'MRN': 'mrn',
    'SurgeryDate': 'date_surgery',
    'VISIT_DATE': 'date_visit',
    'KOOS_JR_L': 'koor_jr_l',
    'KOOS_JR_R': 'koos_jr_r',
    'HOOS_JR_L': 'hoos_jr_l',
    'HOOS_JR_R': 'hoos_jr_r',
    'PROMIS_INTENSITY': 'promis_intensity',
    'PROMIS_INTERFERENCE': 'promis_interference',
    'PROMIS_PHYSICAL_FUNCTION': 'promis_physical_function',
    'PROMIS_MOBILITY': 'promis_mobility',
    'PROMIS_MENTAL_HEALTH': 'promis_mental_health',
    'PROMIS_PHYSICAL_HEALTH': 'promis_physical_health',
}

sheet4.rename(columns=column_names, inplace=True)

In [485]:
# Convert the 'mrn' and 'date_surgery' columns to strings
sheet4['mrn'] = sheet4['mrn'].astype(str)
sheet4['date_surgery'] = sheet4['date_surgery'].astype(str)

# Create a new column by concatenating the 'mrn' and 'date_surgery' columns
sheet4['mrn_date_surgery'] = sheet4['mrn'] + '_' + sheet4['date_surgery']

In [487]:
variables_sheet4 = [
    'koor_jr_l', 'koos_jr_r', 'hoos_jr_l', 'hoos_jr_r', 'promis_intensity', 
    'promis_interference', 'promis_physical_function', 'promis_mobility', 
    'promis_mental_health', 'promis_physical_health'
]

unique_values_data_sheet4 = []

for variable in variables_sheet4:
    unique_values = sheet4[variable].unique()
    unique_values_data_sheet4.append(pd.DataFrame({variable: unique_values}))

value_counts_data_sheet4 = []

for variable in variables_sheet4:
    value_counts = sheet4[variable].value_counts().reset_index()
    value_counts.columns = ['Value', 'Count']
    value_counts_data_sheet4.append(value_counts)

value_counts_df_sheet4 = pd.concat(value_counts_data_sheet4, keys=variables_sheet4, axis=1)
value_counts_df_sheet4 = value_counts_df_sheet4.fillna(value=pd.NA)    
    
unique_values_df_sheet4 = pd.concat(unique_values_data_sheet4, keys=variables_sheet4, axis=1)

# Reset the MultiIndex columns to a single level
unique_values_df_sheet4.columns = ['_'.join(col) for col in unique_values_df_sheet4.columns]
value_counts_df_sheet4.columns = ['_'.join(col) for col in value_counts_df_sheet4.columns]

with pd.ExcelWriter('sheet4_unique_values_and_counts.xlsx') as writer:
    unique_values_df_sheet4.to_excel(writer, sheet_name='Unique Values', index=False)
    value_counts_df_sheet4.to_excel(writer, sheet_name='Value Counts', index=False)


from pandas import ExcelWriter

In [ ]:
writer = ExcelWriter('cleaned_tja_mychart.xlsx', engine='openpyxl')

In [ ]:
sheet1.to_excel(writer, index=False, sheet_name='general')
sheet2.to_excel(writer, index=False, sheet_name='3_weeks_prior')
sheet3.to_excel(writer, index=False, sheet_name='90_days_post')
sheet4.to_excel(writer, index=False, sheet_name='pro')

In [ ]:
writer.save()